In [ ]:
# using the Sentinel API

import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import rasterio

# connect to the API
api = SentinelAPI('username', 'password', 'https://scihub.copernicus.eu/dhus')

# specify date, location
footprint = geojson_to_wkt(read_geojson('path/to/geojson/file'))
products = api.query(footprint,
                     date=(date(2022, 1, 1), date(2022, 1, 31)),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30))

# download the products
api.download_all(products)

# where do we want to download scenes to? PetaLibrary? S3 bucket?

# open with rasterio TO DO

# with rasterio.open(file_path) as src: # etc.
    
# resample Sentinel-2 scenes to 10 m using the bilinear resampling method
# available resampling methods: 
# Resampling.nearest: Nearest neighbor resampling.
# Resampling.bilinear: Bilinear resampling.
# Resampling.cubic: Cubic convolution resampling.
# Resampling.cubic_spline: Cubic spline resampling.
# Resampling.lanczos: Lanczos resampling.
# Resampling.average: Resampling by averaging.
# current and historical scenes should be resampled to 10 m resolution. 
# which resampling method would be prefered in this case?

import rasterio
from rasterio.enums import Resampling

def resample_sentinel2(input_path, output_path, target_resolution):
    with rasterio.open(input_path) as src:
        # Get current resolution/calculate resampling factor
        current_resolution = src.res[0]
        resampling_factor = current_resolution / target_resolution

        # Set variables to calculate new dimensions after resampling
        new_height = int(src.height * resampling_factor)
        new_width = int(src.width * resampling_factor)

        # resampling options - I'm not sure which is the best for this situation
        resampling_method = Resampling.nearest

        # Set resampling parameters
        transform = src.transform * src.transform.scale(
            (src.width / new_width),
            (src.height / new_height)
        )

        # resampling
        resampled_data = src.read(
            out_shape=(src.count, new_height, new_width),
            resampling=resampling_method
        )

        # Update metadata 
        resampled_meta = src.meta
        resampled_meta.update({
            'transform': transform,
            'width': new_width,
            'height': new_height
        })

        # Write to output file
        with rasterio.open(output_path, 'w', **resampled_meta) as dst:
            dst.write(resampled_data)

# On to the next step....TO DO
input_path = 'path/to/input/image.tif'

# gridding

# possibly use co-kriging to interpolate a continuous raster surface with veg and DEM as inputs?

import numpy as np
from pykrige.uk import UniversalKriging

# Load Sentinel-2 data and extract necessary variables
# ...

# Load vegetation and DEM data
# ...

# Co-Kriging
# Combine Sentinel-2, vegetation, and DEM variables into a single feature array
features = np.column_stack((sentinel2_variable, vegetation_variable, dem_variable))

# target coordinates to predict 
target_coordinates = np.array([[x1, y1], [x2, y2], ...])  

# Prepare vegetation class prediction
target_variable = np.array([value1, value2, ...])  

# Co-Kriging
UK = UniversalKriging(features[:, 0], features[:, 1], features[:, 2], target_variable)
z, ss = UK.execute('grid', gridx, gridy)  # use desired grid coordinates

# WIll result in a 'z' array containing predicted values at grid coordinates

# MESMA TO DO - in process in another notebook


# classify TO DO 

